# Assignment

I often need "fake data" to show people how to do data manipulation tasks with regular expressions or pandas. The
problem is that sometimes the data I generate on the web is too messy, and I get bogged down showing students how to
clean all of the data when some of it isn't representative of what I want. In this assignment you get a chance to help
me generate realistic fake data, all with llama 2!

For each question I will describe the data in natural language and you must write a function which queries llama 2 to
generate data in that format and adhere to the description I've written.


In [8]:
import os
import re
from llama_cpp import Llama
from llama_cpp.llama_types import *
from llama_cpp.llama_grammar import *

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

## Question 1

Generate for me a list of ten fictitious names, where the first name is a single word, and the last (family) name may be
(but doesn't have to be!) up to two words separated by a hyphen. Don't include titles, honorifics, or middle names. The
autograder will expect that you return a list[str] where each value in the list is a full name.


In [14]:
def generate_names() -> list[str]:
    # model: Llama = Llama(model_path=os.environ["LLAMA_13B"], verbose=False, n_ctx=2048)
    model: Llama = Llama(
        model_path='models/llama-2-13b.Q4_K_M.gguf',
        verbose=False, 
        n_ctx=2048,
        n_gpu_layers=60, 
        n_threads=16
        )


    results: list[str] = []

    prompt = "Imagine a name and last name like:"
    grammar= r'''
        root ::= [A-Za-z]*
        '''

    result = model.create_completion(prompt,
        grammar=LlamaGrammar.from_string(grammar=grammar), 
        stream=True, 
        max_tokens=16)
    for item in result:
        print(item['choices'][0]['text'], end="")
    
    return results
generate_names()

from_string grammar:
root ::= root_1 
root_1 ::= [A-Za-z] root_1 | 



[]

In [3]:
# Invoke student code
from contextlib import redirect_stderr
import tempfile

with redirect_stderr( tempfile.TemporaryFile('wt') ) as error_catcher:
    results = generate_names()

# Verify the length
assert (
    len(results) == 10
), f"You did not return ten and only ten results, instead we got {len(results)}."


NotImplementedError: 

## Question 2

Generate for me a list of 5 things to do in your hometown (or mine if you prefer, Ann Arbor Michigan!). The key is that
these should all (a) start with a number and (b) be no more than three sentences long. So the following would be a good
item:

- 1\. Go to the Henry Ford Museum. The Henry Ford Museum has all sorts of wonderful exhibits for all ages. One
  particular highlight includes giants trains!

While the following would **not be good items** (the first item does not start a numbered list, the second item is not a
sentence as it doesn't end in punctuation, and the third item just goes on and on and on):

- A\. Go to the University of Michigan. The University of Michigan is a school with more than 50,000 students in Ann
  Arbor, MI. The University of Michigan is a public School.
- 2\. Visit the Detroit Eastern Market
- 3\. Visit Sleeping Bear Dunes. The dunes are located along the northwest coast of the Lower Peninsula of Michigan in
  Leelanau and Benzie counties near Traverse City. It covers a 35-mile-long stretch of Lake Michigan's eastern
  coastline, as well as North and South Manitou islands. This national park is known for its massive dunes, some of
  which are over 400 feet high. The area gets its name from the Native American legend of the Sleeping Bear. According
  to the story, a mother bear and her two cubs were trying to cross Lake Michigan from Wisconsin to escape a forest
  fire.


In [ ]:
def generate_trip_recommendations() -> list[str]:
    results: list[str] = []

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Invoke student code
from contextlib import redirect_stderr
import re
import tempfile

with redirect_stderr( tempfile.TemporaryFile('wt') ) as error_catcher:
    results=generate_trip_recommendations()

# Verify length
assert (
    len(results) == 5
), f"You did not return five and only five results, instead we got {len(results)}."


## Question 3

Generate for me US-based addresses which have a person's name which usually appears on the first line, an optional
company name which often goes on the second line, a street address which has a number followed by some text description,
a city and state where the state is a two letter identifier and comes after the city name, and zip code which is a five
digit number (but as a string, since it could start with 0) followed by an optional hyphen and four more digits.

To make it easy for you to conform to this set of requirements, I created a simple class from the following example --
my mailing address!

> Dr. Christopher Brooks
>
> > School of Information, University of Michigan
> >
> > 105 S. State St.
> >
> > Ann Arbor, MI
> >
> > 48109-1285

Your function should return exactly 5 of these entries!

And, if you've gotten this far in the course, why not send me a postcard and introduce yourself? Everyone loves getting
mail!

(Don't forget to add **United States** if sending mail internationally, even though field is missing from the assignment
`MailingAddress` class.)


In [ ]:
from dataclasses import dataclass


@dataclass
class MailingAddress:
    name: str  # Full name, e.g. Dr. Christopher Brooks
    business_name: (
        str | None
    )  # Optional business name, e.g. School of Information, University of Michigan
    street_number: int  # Numeric address value, e.g. 105
    street_text: str  # Street information other than numeric address, e.g. S. State St.
    city: str  # City name, e.g. Ann Arbor
    state: str  # State name, only two letters, e.g. MI for Michigan
    zip_code_short: str  # The first five digits of the zip code, e.g. 48109, as a string value, since it could start with 0
    zip_code_long: (
        str | None
    )  # The extended zip code (optional) which is the full zip code, e.g. 48109-1285


def generate_addresses() -> list[MailingAddress]:
    results: list[MailingAddress] = []

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Invoke student code
from contextlib import redirect_stderr
import tempfile

with redirect_stderr( tempfile.TemporaryFile('wt') ) as error_catcher:
    results=generate_addresses()

# Verify length
assert (
    len(results) == 5
), f"You did not return five and only five results, instead we got {len(results)}."
